In [422]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
%matplotlib inline

pd.set_option('display.max_columns', 1000000)
pd.set_option('display.max_rows', 1000000)


In [423]:
sal = pd.read_csv('salaries-merged-copy.csv')

In [424]:
sal.head(5)


,Unnamed: 0,function,sup1,sup2,sup3,sup4,sup5,salary,gender,disabled,race,yearsinposition,yearsinprofession,age,cred1,cred2,highestdegree,inst1,inst2,inst3,inst4,inst5,instbudget,city,state,zip,country,category,instsize,insttype,median,mean,pop,b19013_001,no_male_hs_p,no_female_hs_p,at_least_hs_male_p,at_least_hs_female_p,at_least_bach_male_p,at_least_bach_female_p,male_unemployment,female_unemployment,snap,renter,median_rent,foreign_born,value_for_owner-occupied_housing_units:__total:_,"value_for_owner-occupied_housing_units:__less_than_$10,000_","value_for_owner-occupied_housing_units:__$10,000_to_$14,999_","value_for_owner-occupied_housing_units:__$15,000_to_$19,999_","value_for_owner-occupied_housing_units:__$20,000_to_$24,999_","value_for_owner-occupied_housing_units:__$25,000_to_$29,999_","value_for_owner-occupied_housing_units:__$30,000_to_$34,999_","value_for_owner-occupied_housing_units:__$35,000_to_$39,999_","value_for_owner-occupied_housing_units:__$40,000_to_$49,999_","value_for_owner-occupied_housing_units:__$50,000_to_$59,999_","value_for_owner-occupied_housing_units:__$60,000_to_$69,999_","value_for_owner-occupied_housing_units:__$70,000_to_$79,999_","value_for_owner-occupied_housing_units:__$80,000_to_$89,999_","value_for_owner-occupied_housing_units:__$90,000_to_$99,999_","value_for_owner-occupied_housing_units:__$100,000_to_$124,999_","value_for_owner-occupied_housing_units:__$125,000_to_$149,999_","value_for_owner-occupied_housing_units:__$150,000_to_$174,999_","value_for_owner-occupied_housing_units:__$175,000_to_$199,999_","value_for_owner-occupied_housing_units:__$200,000_to_$249,999_","value_for_owner-occupied_housing_units:__$250,000_to_$299,999_","value_for_owner-occupied_housing_units:__$300,000_to_$399,999_","value_for_owner-occupied_housing_units:__$400,000_to_$499,999_","value_for_owner-occupied_housing_units:__$500,000_to_$749,999_","value_for_owner-occupied_housing_units:__$750,000_to_$999,999_","value_for_owner-occupied_housing_units:__$1,000,000_or_more_",sex_by_age:__male:_,sex_by_age:__male:_under_5_years_,sex_by_age:__male:_5_to_9_years_,sex_by_age:__male:_10_to_14_years_,sex_by_age:__male:_15_to_17_years_,sex_by_age:__male:_18_and_19_years_,sex_by_age:__male:_20_years_,sex_by_age:__male:_21_years_,sex_by_age:__male:_22_to_24_years_,sex_by_age:__male:_25_to_29_years_,sex_by_age:__male:_30_to_34_years_,sex_by_age:__male:_35_to_39_years_,sex_by_age:__male:_40_to_44_years_,sex_by_age:__male:_45_to_49_years_,sex_by_age:__male:_50_to_54_years_,sex_by_age:__male:_55_to_59_years_,sex_by_age:__male:_60_and_61_years_,sex_by_age:__male:_62_to_64_years_,sex_by_age:__male:_65_and_66_years_,sex_by_age:__male:_67_to_69_years_,sex_by_age:__male:_70_to_74_years_,sex_by_age:__male:_75_to_79_years_,sex_by_age:__male:_80_to_84_years_,sex_by_age:__male:_85_years_and_over_,sex_by_age:__female:_,sex_by_age:__female:_under_5_years_,sex_by_age:__female:_5_to_9_years_,sex_by_age:__female:_10_to_14_years_,sex_by_age:__female:_15_to_17_years_,sex_by_age:__female:_18_and_19_years_,sex_by_age:__female:_20_years_,sex_by_age:__female:_21_years_,sex_by_age:__female:_22_to_24_years_,sex_by_age:__female:_25_to_29_years_,sex_by_age:__female:_30_to_34_years_,sex_by_age:__female:_35_to_39_years_,sex_by_age:__female:_40_to_44_years_,sex_by_age:__female:_45_to_49_years_,sex_by_age:__female:_50_to_54_years_,sex_by_age:__female:_55_to_59_years_,sex_by_age:__female:_60_and_61_years_,sex_by_age:__female:_62_to_64_years_,sex_by_age:__female:_65_and_66_years_,sex_by_age:__female:_67_to_69_years_,sex_by_age:__female:_70_to_74_years_,sex_by_age:__female:_75_to_79_years_,sex_by_age:__female:_80_to_84_years_,sex_by_age:__female:_85_years_and_over_,commute_over_60,full_time,part_time,high_rent_burden,extreme_rent_burden
0,2211,position22,0,0,0,4,1,49088,Man,False,Caucasian,1,1,NaN,False,False,Bachelor's Degree,9,0,2,60,0,1250000,Fairbanks,AK,99775,United States,one,9223,Public Institution,"79,357","120,734","1,251",86579,0,0,100,100,75.1773

In [425]:
#Group columns by data type for getting dummy variables
datatypes = sal.columns.to_series().groupby(sal.dtypes).groups

In [426]:
#Remove missing data
sal['missingSalary'] = pd.isnull(sal['salary'])
sal = sal[(sal.missingSalary == False)]

In [427]:
#replace alternate Canada spelling
sal["country"] = sal["country"].replace("Canada", "CANADA")

In [428]:
#Prepare variables for reshaping
X_numeric_features = ['disabled', 'cred1', 'cred2', 'Unnamed: 0',
  'sup1',
  'sup2',
  'sup3',
  'sup4',
  'sup5',
  'inst1',
  'inst2',
  'inst3',
  'inst4',
  'inst5',
  'zip',
  'yearsinposition',
  'yearsinprofession',
  'age',
  'instbudget',
  'instsize',
  'b19013_001',
  'no_male_hs_p',
  'no_female_hs_p',
  'at_least_hs_male_p',
  'at_least_hs_female_p',
  'at_least_bach_male_p',
  'at_least_bach_female_p',
  'male_unemployment',
  'female_unemployment',
  'snap',
  'renter',
  'median_rent',
  'foreign_born',
  'value_for_owner-occupied_housing_units:__total:_',
  'value_for_owner-occupied_housing_units:__less_than_$10,000_',
  'value_for_owner-occupied_housing_units:__$10,000_to_$14,999_',
  'value_for_owner-occupied_housing_units:__$15,000_to_$19,999_',
  'value_for_owner-occupied_housing_units:__$20,000_to_$24,999_',
  'value_for_owner-occupied_housing_units:__$25,000_to_$29,999_',
  'value_for_owner-occupied_housing_units:__$30,000_to_$34,999_',
  'value_for_owner-occupied_housing_units:__$35,000_to_$39,999_',
  'value_for_owner-occupied_housing_units:__$40,000_to_$49,999_',
  'value_for_owner-occupied_housing_units:__$50,000_to_$59,999_',
  'value_for_owner-occupied_housing_units:__$60,000_to_$69,999_',
  'value_for_owner-occupied_housing_units:__$70,000_to_$79,999_',
  'value_for_owner-occupied_housing_units:__$80,000_to_$89,999_',
  'value_for_owner-occupied_housing_units:__$90,000_to_$99,999_',
  'value_for_owner-occupied_housing_units:__$100,000_to_$124,999_',
  'value_for_owner-occupied_housing_units:__$125,000_to_$149,999_',
  'value_for_owner-occupied_housing_units:__$150,000_to_$174,999_',
  'value_for_owner-occupied_housing_units:__$175,000_to_$199,999_',
  'value_for_owner-occupied_housing_units:__$200,000_to_$249,999_',
  'value_for_owner-occupied_housing_units:__$250,000_to_$299,999_',
  'value_for_owner-occupied_housing_units:__$300,000_to_$399,999_',
  'value_for_owner-occupied_housing_units:__$400,000_to_$499,999_',
  'value_for_owner-occupied_housing_units:__$500,000_to_$749,999_',
  'value_for_owner-occupied_housing_units:__$750,000_to_$999,999_',
  'value_for_owner-occupied_housing_units:__$1,000,000_or_more_',
  'sex_by_age:__male:_',
  'sex_by_age:__male:_under_5_years_',
  'sex_by_age:__male:_5_to_9_years_',
  'sex_by_age:__male:_10_to_14_years_',
  'sex_by_age:__male:_15_to_17_years_',
  'sex_by_age:__male:_18_and_19_years_',
  'sex_by_age:__male:_20_years_',
  'sex_by_age:__male:_21_years_',
  'sex_by_age:__male:_22_to_24_years_',
  'sex_by_age:__male:_25_to_29_years_',
  'sex_by_age:__male:_30_to_34_years_',
  'sex_by_age:__male:_35_to_39_years_',
  'sex_by_age:__male:_40_to_44_years_',
  'sex_by_age:__male:_45_to_49_years_',
  'sex_by_age:__male:_50_to_54_years_',
  'sex_by_age:__male:_55_to_59_years_',
  'sex_by_age:__male:_60_and_61_years_',
  'sex_by_age:__male:_62_to_64_years_',
  'sex_by_age:__male:_65_and_66_years_',
  'sex_by_age:__male:_67_to_69_years_',
  'sex_by_age:__male:_70_to_74_years_',
  'sex_by_age:__male:_75_to_79_years_',
  'sex_by_age:__male:_80_to_84_years_',
  'sex_by_age:__male:_85_years_and_over_',
  'sex_by_age:__female:_',
  'sex_by_age:__female:_under_5_years_',
  'sex_by_age:__female:_5_to_9_years_',
  'sex_by_age:__female:_10_to_14_years_',
  'sex_by_age:__female:_15_to_17_years_',
  'sex_by_age:__female:_18_and_19_years_',
  'sex_by_age:__female:_20_years_',
  'sex_by_age:__female:_21_years_',
  'sex_by_age:__female:_22_to_24_years_',
  'sex_by_age:__female:_25_to_29_years_',
  'sex_by_age:__female:_30_to_34_years_',
  'sex_by_age:__female:_35_to_39_years_',
  'sex_by_age:__female:_40_to_44_years_',
  'sex_by_age:__female:_45_to_49_years_',
  'sex_by_age:__female:_50_to_54_years_',
  'sex_by_age:__female:_55_to_59_years_',
  'sex_by_age:__female:_60_and_61_years_',
  'sex_by_age:__female:_62_to_64_years_',
  'sex_by_age:__female:_65_and_66_years_',
  'sex_by_age:__female:_67_to_69_years_',
  'sex_by_age:__female:_70_to_74_years_',
  'sex_by_age:__female:_75_to_79_years_',
  'sex_by_age:__female:_80_to_84_years_',
  'sex_by_age:__female:_85_years_and_over_',
  'commute_over_60',
  'full_time',
  'part_time',
  'high_rent_burden',
  'extreme_rent_burden']
X_numeric = sal[X_numeric_features]
X_categorical_features = ['function', 'gender', 'race', 'highestdegree', 'category', 'insttype', 'city', 'state', 'country', 'median', 'mean', 'pop']
X_categorical = sal[X_categorical_features]





In [429]:
#Concatenate dummy variables into a matrix
Function_dummies = pd.get_dummies(X_categorical['function'])
gender_dummies = pd.get_dummies(X_categorical['gender'])
race_dummies = pd.get_dummies(X_categorical['race'])
highestDegree_dummies = pd.get_dummies(X_categorical['highestdegree'])
Category_dummies = pd.get_dummies(X_categorical['category'])
instType_dummies = pd.get_dummies(X_categorical['insttype'])
city_dummies = pd.get_dummies(X_categorical['city'])
state_dummies = pd.get_dummies(X_categorical['state'])
country_dummies = pd.get_dummies(X_categorical['country'])
median_dummies = pd.get_dummies(X_categorical['median'])
mean_dummies = pd.get_dummies(X_categorical['mean'])
pop_dummies = pd.get_dummies(X_categorical['pop'])

X_dummy_features = pd.concat([Function_dummies, gender_dummies, race_dummies, highestDegree_dummies, Category_dummies, instType_dummies, city_dummies, 
state_dummies,
country_dummies,
median_dummies,
mean_dummies,
pop_dummies], axis=1)

# convert to ndarray
X_dummy_features = X_dummy_features.as_matrix()

In [430]:
#Impute values
from sklearn.preprocessing import Imputer
imp = Imputer()
imp.fit(X_numeric)
X_numeric_imputed = imp.transform(X_numeric)

In [431]:
#Combine final X values
X = np.concatenate((X_dummy_features, X_numeric_imputed), axis=1)
#X = np.expand_dims(X, axis=1)
y = sal.iloc[:, 7].values
y = np.expand_dims(y, axis=1)

In [432]:
#Split data for training and test
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [433]:
#Scale data
from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler().fit(X_train)
scalery = StandardScaler().fit(y_train)
X_train = scalerX.transform(X_train)
y_train = scalery.transform(y_train)
X_test = scalerX.transform(X_test)
y_test = scalery.transform(y_test)

In [434]:
#Reshape Y
y_train = y_train.reshape(1529,)

In [435]:
'''create a function to train and cross-validate the data using:
SGD regressor; 
SVR with Linear, Poly, and RBF kernels; 
extra-trees regressor;
Linear regression'''
from sklearn.cross_validation import *
def train_and_evaluate(regr, X_train, y_train, name):
    regr.fit(X_train, y_train)
    print("Training and cross-validating data using: ", name)
    print(" ")
    print ("Variance on training set:\n",regr.score(X_train, y_train))
        # create a k-fold cross validation iterator of k=10 folds
    print(" ")
    print("Use K-fold cross-validation to validate model prediction")
    cv = KFold(X_train.shape[0], 10, shuffle=True, random_state=42)
    scores = cross_val_score(regr, X_train, y_train, cv=cv)
    print(scores)
    print(" ")   
    #Average the 10 folds
    print ("Average Variance using 10-fold cross validation:",np.mean(scores))
    print(" ")   
        # The mean square error
    print("Residual sum of squares, training data: %.2f" % np.mean((regr.predict(X_train) - y_train) ** 2))

In [436]:
from sklearn import linear_model

In [437]:
clf_sgd = linear_model.SGDRegressor(loss='squared_loss', penalty=None,  random_state=42)
train_and_evaluate(clf_sgd,X_train,y_train, 'SGD No Penalty')


Training and cross-validating data using:  SGD No Penalty
 
Variance on training set:
 -1.42836924383e+13
 
Use K-fold cross-validation to validate model prediction
[ -1.82910636e+09  -9.16001161e+11  -9.75230642e+10  -2.55930368e+11
  -2.61379921e+10  -8.93926908e+08  -4.19740224e+09  -1.63738420e+11
  -2.09056841e+10  -8.32051442e+10]
 
Average Variance using 10-fold cross validation: -157036226932.0
 
Residual sum of squares, training data: 14283692438278.80


In [438]:
#Adding penalty to SGD
clf_sgd = linear_model.SGDRegressor(loss='squared_loss', penalty='l1',  random_state=42)
train_and_evaluate(clf_sgd, X_train, y_train, "SGD regressor with penalty")

Training and cross-validating data using:  SGD regressor with penalty
 
Variance on training set:
 -1.42656827401e+13
 
Use K-fold cross-validation to validate model prediction
[ -1.80158553e+09  -9.12822932e+11  -9.75263047e+10  -2.55570811e+11
  -2.60266074e+10  -8.79754793e+08  -4.11771498e+09  -1.63365299e+11
  -2.07282925e+10  -7.98055256e+10]
 
Average Variance using 10-fold cross validation: -156264482664.0
 
Residual sum of squares, training data: 14265682740118.68


In [439]:
from sklearn import svm

In [440]:
clf_svr = svm.SVR(kernel='linear')
train_and_evaluate(clf_svr, X_train, y_train, "Support vector regression linear kernel")


Training and cross-validating data using:  Support vector regression linear kernel
 
Variance on training set:
 0.817293686151
 
Use K-fold cross-validation to validate model prediction
[ 0.74350105  0.81939927  0.69438712  0.74716226  0.75200189  0.78439972
  0.70416087  0.70247894  0.69142948  0.81289127]
 
Average Variance using 10-fold cross validation: 0.745181186174
 
Residual sum of squares, training data: 0.18


In [441]:
clf_svr_poly = svm.SVR(kernel='poly')
train_and_evaluate(clf_svr_poly, X_train, y_train, "Support vector regression poly kernel")

Training and cross-validating data using:  Support vector regression poly kernel
 
Variance on training set:
 0.28916476719
 
Use K-fold cross-validation to validate model prediction
[-0.02665512 -0.01057809  0.05490128  0.01170454 -0.01067613 -0.0492374
 -0.07955932 -0.04096236  0.11776731  0.15299641]
 
Average Variance using 10-fold cross validation: 0.0119701112025
 
Residual sum of squares, training data: 0.71


In [442]:
clf_svr_rbf = svm.SVR(kernel='rbf')
train_and_evaluate(clf_svr_rbf, X_train, y_train, "Support vector regression rbf kernel")

Training and cross-validating data using:  Support vector regression rbf kernel
 
Variance on training set:
 0.723760938372
 
Use K-fold cross-validation to validate model prediction
[ 0.58973847  0.67441921  0.53563176  0.56841997  0.58537415  0.71335795
  0.49498322  0.43537103  0.59316982  0.64657685]
 
Average Variance using 10-fold cross validation: 0.583704241422
 
Residual sum of squares, training data: 0.28


In [443]:
from sklearn import ensemble
clf_et=ensemble.ExtraTreesRegressor(n_estimators=10, random_state=42)
train_and_evaluate(clf_et, X_train, y_train, "Extra trees regressor")

Training and cross-validating data using:  Extra trees regressor
 
Variance on training set:
 0.999999999861
 
Use K-fold cross-validation to validate model prediction
[ 0.77944728  0.80892385  0.72328764  0.75769275  0.69151912  0.79844592
  0.74930601  0.71109228  0.70623287  0.77019544]
 
Average Variance using 10-fold cross validation: 0.749614315378
 
Residual sum of squares, training data: 0.00


In [444]:
regr = linear_model.LinearRegression()
train_and_evaluate(regr, X_train, y_train, "Linear regression")

Training and cross-validating data using:  Linear regression
 
Variance on training set:
 0.833701933653
 
Use K-fold cross-validation to validate model prediction
[ -9.02770460e+24  -3.05062409e+25  -9.11405264e+25  -8.55413433e+25
  -6.22215690e+23  -5.53316719e+25  -1.51623487e+26  -2.02911537e+26
  -1.28411050e+26  -4.02540585e+24]
 
Average Variance using 10-fold cross validation: -7.59141181882e+25
 
Residual sum of squares, training data: 0.17


In [445]:
#Create function to run trained models on test data
from sklearn import metrics
def measure_performance(X, y, clf, name, show_r2_score=False):
    y_pred = clf.predict(X)
    if show_r2_score:
        print("Predicting test-set using: ", name)
        print(" ")
        print ("Variance for test set:{0:.5f}".format(metrics.r2_score(y, y_pred)))

In [446]:
measure_performance(X_test, y_test, regr, "Linear regression", show_r2_score=True)

Predicting test-set using:  Linear regression
 
Variance for test set:-2202619156856272261742592.00000


In [447]:
measure_performance(X_test, y_test, clf_sgd, "SGD kernel", show_r2_score=True)

Predicting test-set using:  SGD kernel
 
Variance for test set:-3473745428577.87158


In [448]:
measure_performance(X_test, y_test, clf_svr, "SVR linear kernel", show_r2_score=True)

Predicting test-set using:  SVR linear kernel
 
Variance for test set:0.72829


In [449]:
measure_performance(X_test, y_test, clf_svr_poly, "SVR poly kernel", show_r2_score=True)

Predicting test-set using:  SVR poly kernel
 
Variance for test set:0.10094


In [450]:
measure_performance(X_test, y_test, clf_svr_rbf, "SVR rbf kernel", show_r2_score=True)

Predicting test-set using:  SVR rbf kernel
 
Variance for test set:0.57954


In [451]:
measure_performance(X_test, y_test, clf_et, "Extra-Trees regressor", show_r2_score=True)

Predicting test-set using:  Extra-Trees regressor
 
Variance for test set:0.70555
